<hr>

#1 Izracunavanje bajt-entropije datog binarnog fajla
- Na pocetku procitamo sadrzaj binarnog fajla
- Odredimo duzinu N
- Odredimo broj pojavljivanja svakog bajta Ni
- Izracunamo verovatnocu p = Ni/N
- Ispisemo koji bajt se koliko puta pojavljuje i njegova verovatnoca
- Racunamo bajt-Entropiju H(p) = −p0 log2 p0 − p1 log2 p1 − . . . − p255 log2 p255
- Cuvamo vrednost entropije u kodiranom fajlu

In [5]:
import math
import os
import filecmp

trenutni_direktoriju = os.getcwd()
putanja = os.path.join(trenutni_direktoriju, 'Ulazni_alfabet_Shanno_Fano_i_Huff.bin')

karakteri_niz = []
verovatnoca_niz = []

def racunanje_Bajt_Entropije(putanja):
    # Otvori binarni fajl u modu za čitanja
    with open(putanja, 'rb') as fajl:
        # Čitanje sadržaja fajla
        sadrzaj_Fajla = fajl.read()
    
    # N-> Ukupna duzina fajla u bajtovima
    N = len(sadrzaj_Fajla)

    # Brojanje pojavljivanja svakog bajta
    bajt_brojac = [sadrzaj_Fajla.count(i) for i in range(256)]

    # p-> verovatnoca, ovde racunamo verovatnocu svakog bajta
    p = [brojac / N for brojac in bajt_brojac] # p = Ni/N

    # Ispisivanje brojanja pojavljivanja svakog bajta
    for byte, (brojac, p_i) in enumerate(zip(bajt_brojac, p)):
        if brojac > 0:
             karakteri = chr(byte)
             karakteri_niz.append(karakteri)
             verovatnoca_niz.append(p_i)
             print(f'{karakteri}: Pojavljuje se {brojac} puta, Verovatnoca: {p_i:.2f}')
    
    # Izračunavanje bajt-entropije
    entropija = -sum(p_i * math.log2(p_i) if p_i != 0 else 0 for p_i in p)

    return entropija,sadrzaj_Fajla

rezultat_Entropije,ulazni_alfabet = racunanje_Bajt_Entropije(putanja)
print(f'Bajt-entropija fajla je: {rezultat_Entropije:.2f}')

1: Pojavljuje se 461014 puta, Verovatnoca: 0.31
2: Pojavljuje se 346450 puta, Verovatnoca: 0.23
3: Pojavljuje se 346503 puta, Verovatnoca: 0.23
4: Pojavljuje se 230652 puta, Verovatnoca: 0.15
5: Pojavljuje se 115381 puta, Verovatnoca: 0.08
Bajt-entropija fajla je: 2.20


<hr>

#2.1.  Konstruisati Shannon-Fano kod - KODIRANJE
- Na samom pocetku potrebno je sortirati simbole ulaznog alfabeta po verovatnoci
- Potrebno je simbole podelili na 2 grupe, ali da te dve grupe budu u sto boljoj ravnotezi gledajuci verovatnocu
- Ako se simbol nalazi u prvoj grupi dobija kod 0.
- Ako se simbol nalazi u drugoj grupi dobija kod 1.
- Postupak se ponavlja rekurzivno sve dok u grupi ne ostane 1 simbol

#Zatim sledi faza DEKODIRANJA
- Na samom pocetnu smo iz prvog reda izvukli simbole da dekodiranje
- A zatim u niz, bajtove pretvorili u niz bitova koja je potrebno dekodirati
- Zatim sledi dekodiranje gde smo za svaki kod pronasli simbol koji mu odgovara
- Ispis rezultata

In [7]:
class Simbol: 
    def __init__(self, vrednost, verovatnoca):
        self.vrednost = vrednost
        self.verovatnoca = verovatnoca
        self.kod = ""

# Konstruisemo kod
def shannon_fano_kodiranje(simboli):
    if len(simboli) == 1: #Uslov da rekurzija stane je da postoji 1 simbol u grupi
        return

    # Na dalje trazimo simbol koji predstavlja granicu optimalne podele
    suma_verovatnoci = sum(simbol.verovatnoca for simbol in simboli) 

    trenutna_suma = 0
    razlika = []

    for i, simbol in enumerate(simboli):
        trenutna_suma += simbol.verovatnoca
        ostatak = suma_verovatnoci - trenutna_suma
        razlika.append(round(abs(trenutna_suma - ostatak), 3))

    element_za_deljenje = min(razlika)
    indeks_elementa = razlika.index(element_za_deljenje)

    for i, simbol in enumerate(simboli):
        if i <= indeks_elementa:
            simbol.kod += "0"
        else:
            simbol.kod += "1"

    #Pozivamo rekurzivno prvu polovinu niza i drugu
    shannon_fano_kodiranje(simboli[:indeks_elementa + 1])
    shannon_fano_kodiranje(simboli[indeks_elementa + 1:])

#Prikaz dobijenog koda
def prikazi_kodove(simboli):
    for simbol in simboli:
        print(f"Simbol {simbol.vrednost}: {simbol.kod}")
    
# Mapiranje simbola na odgovarajuće Shannon-Fano kodove
def kodiraj_niz(niz, simboli):
    kod_niza = ""
    for simbol in niz:
        for s in simboli:
            if s.vrednost == simbol:
                kod_niza += s.kod
                break
    return kod_niza

# Kreiranje liste Simbol objekata
simboli = [Simbol(karakteri_niz[i], verovatnoca_niz[i]) for i in range(len(karakteri_niz))]

# Sortiranje simbola po opadajućoj verovatnoći
simboli.sort(key=lambda x: x.verovatnoca, reverse=True)

# Primena Shannon-Fano kodiranja
shannon_fano_kodiranje(simboli)

print("KODIRANJE:\n")
# Prikaz rezultata
prikazi_kodove(simboli)

# Kodiranje ulaznog alfabeta
kod_niza = kodiraj_niz(ulazni_alfabet.decode('utf-8'), simboli)

# print(f"Kodiran niz: {kod_niza}")
duzina = len(kod_niza)


bajtovi = bytearray()
for i in range(0,len(kod_niza),8):
    bajt = int(kod_niza[i:i+8],2)
    bajtovi.append(bajt)

with open("Kodirani_fajl_shannon_fano.bin","wb") as f:
    for simbol in simboli:
        informacija_o_simbolu = f"{simbol.vrednost}:{simbol.kod}"
        f.write(informacija_o_simbolu.encode("utf-8"))
        f.write(b' ')
    f.write(b"\n")    
    f.write(bajtovi)

class UcitaniSimbol: 
    def __init__(self, vrednost, kod):  # Ispravljeno mesto vrednosti i koda
        self.vrednost = vrednost
        self.kod = kod

def ucitaj_prvi_red_iz_binarnog_fajla_Shanno_Fano(putanja):
    with open(putanja, 'rb') as fajl:
        prvi_red_bin = fajl.readline().decode('utf-8').strip()  # Čita prvi red iz binarnog fajla i dekodira ga u string
        simboli_raw = prvi_red_bin.split()  # Razdvaja string na osnovu razmaka
        simboli = []

        for simbol_raw in simboli_raw:
            vrednost, kod = simbol_raw.split(":")
            simbol = UcitaniSimbol(vrednost, kod)
            simboli.append(simbol)

    return simboli

def ucitaj_ostatak_iz_binarnog_fajla_Shanno_Fano(putanja):
    ostatak  = duzina % 8

    with open(putanja, 'rb') as fajl:
        # Preskoči prvi red
        fajl.readline()

        # Čitaj ostatak fajla
        podaci = fajl.read()

    # Konvertuj binarne podatke u niz bitova
    niz_bita = []
    for bajt in podaci:
        bitovi = bin(bajt)[2:].rjust(8, '0')
        niz_bita.extend(map(int, bitovi))

    # Spoji niz bitova u string bez zareza
    bitni_string = ''.join(map(str, niz_bita))

    if ostatak > 0:
        fali = 8 - ostatak
        pomocni_niz = bitni_string[-8:]
        bitni_string = bitni_string[:-8]
        pomocni_niz = pomocni_niz[fali:]
        bitni_string += pomocni_niz

    return bitni_string

def dekodiraj(niz, simboli):
    dekodirani_niz = ""
    trenutni_kod = ""
    
    for bit in niz:
        trenutni_kod += bit
        for simbol in simboli:
            if trenutni_kod == simbol.kod:
                dekodirani_niz += simbol.vrednost
                trenutni_kod = ""
                break  # Prekidamo unutrašnju petlju jer smo pronašli odgovarajući simbol

    return dekodirani_niz

fajl_za_dekodiranje = os.path.join(trenutni_direktoriju, 'Kodirani_fajl_shannon_fano.bin')
simboli_iz_binarnog_fajla = ucitaj_prvi_red_iz_binarnog_fajla_Shanno_Fano(fajl_za_dekodiranje)
sadrzaj_za_dekodiranje = ucitaj_ostatak_iz_binarnog_fajla_Shanno_Fano(fajl_za_dekodiranje)

print("\nDEKODIRANJE:\n")
# Ispisivanje učitanih simbola
for simbol in simboli_iz_binarnog_fajla:
    print(f"Simbol: {simbol.vrednost}: {simbol.kod}")

dekodirani_niz = dekodiraj(sadrzaj_za_dekodiranje, simboli_iz_binarnog_fajla)

ulazni_Primer = os.path.join(trenutni_direktoriju, 'Ulazni_alfabet_Shanno_Fano_i_Huff.bin')
dekodiran_PrimerShanno = os.path.join(trenutni_direktoriju, 'DekodiranShano.bin')

with open(dekodiran_PrimerShanno, 'wb') as fajl:
    podaci = dekodirani_niz.encode('utf-8')
    fajl.write(podaci)


if filecmp.cmp(ulazni_Primer,dekodiran_PrimerShanno):
    print("\nFajlovi imaju isti sadržaj.")
else:
    print("\nFajlovi nemaju isti sadržaj.")


KODIRANJE:

Simbol 1: 00
Simbol 3: 01
Simbol 2: 10
Simbol 4: 110
Simbol 5: 111

DEKODIRANJE:

Simbol: 1: 00
Simbol: 3: 01
Simbol: 2: 10
Simbol: 4: 110
Simbol: 5: 111

Fajlovi imaju isti sadržaj.


<hr>

#2.2. Konstruisati Huffmanov kod - KODIRANJE
- Definisemo strukturu jednog cvora za izgradnju stabla  
- Pravimo stablo prema Hafmanovom algoritmu:
    - Potrebno je prvo sortirati prema vrednosti verovatnoca
    - Trazimo 2 cvora sa najmanjom verovatnocom
    - Njih brisemo, i pravimo novi roditeljski cvor cija je vrednost verovatnoce zbir 2 cvora sa najmanjom verovatnocom
    - Postupak ponavljamo
- Zatim ispisujemo dobijene kodne reci
- Cuvamo dobijene kodne reci
- Kodiramo binarni fajl sa dobijenim kodnim recima
- Cuvamo dobijeni kod

#Zatim sledi faza DEKODIRANJA
- Na samom pocetnu smo iz prvog reda izvukli simbole da dekodiranje
- A zatim u niz, bajtove pretvorili u niz bitova koja je potrebno dekodirati
- Zatim sledi dekodiranje gde smo za svaki kod pronasli simbol koji mu odgovara
- Ispis rezultata


In [8]:
class Cvor:
    def __init__(self, verovatnoca, simbol, levo=None, desno=None):
        self.verovatnoca = verovatnoca
        self.simbol = simbol
        self.levo = levo
        self.desno = desno
        self.kod = ''

#Pravimo Haffanovo stablo
def izgradi_hafmanovo_stablo(karakteri_niz, verovatnoca_niz):
    #Kreiranje liste cvorova
    cvorovi = [Cvor(verovatnoca_niz[i], karakteri_niz[i]) for i in range(len(karakteri_niz))]

    while len(cvorovi) > 1:
        #Sortiramo po vrednosti
        cvorovi.sort(key=lambda x: x.verovatnoca)
        #Izbacujemo 2 cvora sa najmanjom verovatnocom
        levo = cvorovi.pop(0)
        desno = cvorovi.pop(0)
        #Dodeljujemo im kod
        levo.kod = 0
        desno.kod = 1
        #Pravimo nov cvor koji predstavlja zbir verovatnoca koja su imala dve najmanje verovatnoce
        novi_cvor = Cvor(levo.verovatnoca + desno.verovatnoca, levo.simbol + desno.simbol, levo, desno)
        cvorovi.append(novi_cvor)

    return cvorovi[0]

#Ispis dobijenih kodnih reci
def ispisi_cvorove(cvor, vrednost=''):
    nova_vrednost = vrednost + str(cvor.kod)
    
    if cvor.levo:
        ispisi_cvorove(cvor.levo, nova_vrednost)
    if cvor.desno:
        ispisi_cvorove(cvor.desno, nova_vrednost)

    if not cvor.levo and not cvor.desno:
        print(f"{cvor.simbol} -> {nova_vrednost}")

#Kodiranje binarnog fajla   
def kodiraj_niz(niz, koren):
    kod_niza = ""

    for karakter in niz:
        kod_niza += pronadji_kod(karakter, koren)

    return kod_niza

#Trazimo za taj simbol koja je kod
def pronadji_kod(simbol, cvor, trenutni_kod=""):
    if not cvor:
        return None

    if cvor.simbol == simbol:
        return trenutni_kod
    else:
        levi_kod = pronadji_kod(simbol, cvor.levo, trenutni_kod + "0")
        desni_kod = pronadji_kod(simbol, cvor.desno, trenutni_kod + "1")

        if levi_kod:
            return levi_kod
        elif desni_kod:
            return desni_kod

#Cuvanje cvorova u fajlu
def cuvaj_cvorove_u_stringu(cvor, vrednost=''):
    nova_vrednost = vrednost + str(cvor.kod)
    
    informacije_o_cvorovima = ""

    if cvor.levo:
        informacije_o_cvorovima += cuvaj_cvorove_u_stringu(cvor.levo, nova_vrednost)
    if cvor.desno:
        informacije_o_cvorovima += cuvaj_cvorove_u_stringu(cvor.desno, nova_vrednost)

    if not cvor.levo and not cvor.desno:
        informacije_o_cvorovima += f"{cvor.simbol}:{nova_vrednost} "

    return informacije_o_cvorovima


# Sortiranje cvorova na pocetku prema verovatnoci u opadajućem redosledu
cvorovi = [Cvor(verovatnoca_niz[i], karakteri_niz[i]) for i in range(len(karakteri_niz))]
cvorovi.sort(key=lambda x: x.verovatnoca, reverse=True)

#Pravimo stabnlo i dobijamo kodne reci
koren_hafmanovog_stabla = izgradi_hafmanovo_stablo(karakteri_niz, verovatnoca_niz)
print("KODIRANJE:\n")
print("Huffmanove kodne reci:")
ispisi_cvorove(koren_hafmanovog_stabla)

#Kodiramo ulazni alfabet
kodiran_niz = kodiraj_niz(ulazni_alfabet.decode('utf-8'), koren_hafmanovog_stabla)
duzinaHuff = len(kodiran_niz)
# print(f"Kodirani niz: \n{kodiran_niz}")

bajtovi = bytearray()
for i in range(0,len(kodiran_niz),8):
    bajt = int(kodiran_niz[i:i+8],2)
    bajtovi.append(bajt)

with open("Kodiran_fajl_huff.bin", "wb") as fajl:
    informacije_o_cvorovima = cuvaj_cvorove_u_stringu(koren_hafmanovog_stabla)
    fajl.write(informacije_o_cvorovima.encode("utf-8"))
    fajl.write(b"\n")
    fajl.write(bajtovi)

class UcitaniSimbol: 
    def __init__(self, vrednost, kod):  # Ispravljeno mesto vrednosti i koda
        self.vrednost = vrednost
        self.kod = kod

def ucitaj_prvi_red_iz_binarnog_fajla_Huffman(putanja):
    with open(putanja, 'rb') as fajl:
        prvi_red_bin = fajl.readline().decode('utf-8').strip()  # Čita prvi red iz binarnog fajla i dekodira ga u string
        simboli_raw = prvi_red_bin.split()  # Razdvaja string na osnovu razmaka
        simboli = []

        for simbol_raw in simboli_raw:
            vrednost, kod = simbol_raw.split(":")
            simbol = UcitaniSimbol(vrednost, kod)
            simboli.append(simbol)

    return simboli

def ucitaj_ostatak_iz_binarnog_fajla__Huffman(putanja):
    ostatak  = duzinaHuff % 8

    with open(putanja, 'rb') as fajl:
        # Preskoči prvi red
        fajl.readline()

        # Čitaj ostatak fajla
        podaci = fajl.read()

    # Konvertuj binarne podatke u niz bitova
    niz_bita = []
    for bajt in podaci:
        bitovi = bin(bajt)[2:].rjust(8, '0')
        niz_bita.extend(map(int, bitovi))

    # Spoji niz bitova u string bez zareza
    bitni_string = ''.join(map(str, niz_bita))

    if ostatak > 0:
        fali = 8 - ostatak
        pomocni_niz = bitni_string[-8:]
        bitni_string = bitni_string[:-8]
        pomocni_niz = pomocni_niz[fali:]
        bitni_string += pomocni_niz

    return bitni_string

def dekodiraj_Huff(niz, simboli):
    dekodirani_niz = ""
    trenutni_kod = ""
    
    for bit in niz:
        trenutni_kod += bit
        for simbol in simboli:
            if trenutni_kod == simbol.kod:
                dekodirani_niz += simbol.vrednost
                trenutni_kod = ""
                break  # Prekidamo unutrašnju petlju jer smo pronašli odgovarajući simbol

    return dekodirani_niz

fajl_za_dekodiranje = os.path.join(trenutni_direktoriju, 'Kodiran_fajl_huff.bin')
simboli_iz_binarnog_fajla = ucitaj_prvi_red_iz_binarnog_fajla_Huffman(fajl_za_dekodiranje)
sadrzaj_za_dekodiranje = ucitaj_ostatak_iz_binarnog_fajla__Huffman(fajl_za_dekodiranje)

print("\nDEKODIRANJE:\n")
# Ispisivanje učitanih simbola
for simbol in simboli_iz_binarnog_fajla:
    print(f"Simbol: {simbol.vrednost}: {simbol.kod}")

dekodirani_niz = dekodiraj_Huff(sadrzaj_za_dekodiranje, simboli_iz_binarnog_fajla)

ulazni_Primer = os.path.join(trenutni_direktoriju, 'Ulazni_alfabet_Shanno_Fano_i_Huff.bin')
dekodiran_PrimerHuff = os.path.join(trenutni_direktoriju, 'DekodiranHuff.bin')

with open(dekodiran_PrimerHuff, 'wb') as fajl:
    podaci = dekodirani_niz.encode('utf-8')
    fajl.write(podaci)


if filecmp.cmp(ulazni_Primer,dekodiran_PrimerHuff):
    print("\nFajlovi imaju isti sadržaj.")
else:
    print("\nFajlovi nemaju isti sadržaj.")


KODIRANJE:

Huffmanove kodne reci:
5 -> 000
4 -> 001
2 -> 01
3 -> 10
1 -> 11

DEKODIRANJE:

Simbol: 5: 000
Simbol: 4: 001
Simbol: 2: 01
Simbol: 3: 10
Simbol: 1: 11

Fajlovi imaju isti sadržaj.


<hr>

#3.1 LZ77 - KODIRANJE
- Postavili smo da je velicina prozora (windows size = 4), odosno koliko mesta unazad gledamo da li ima poklapanja
- Prolazimo kroz ceo niz, pomocu while petlje
- Pomocu unutrasnje petlje (for j) proveravamo poklapanja poredeci podnizove ulaznog niza
- Zatim proveravamo poklapanje i pisemo izlaz
    - Ako je pronadjeno poklapanje (maks_duzina_poklapanja > 0), dodaje tuple (1, pomeraj, duzina) u kompresovani izlaz.
        - 1 označava poklapanje
        - pomeraj - je rastojanje unazad do početka poklapanja
        - duzina je dužina poklapanja
    - Ako nema poklapanja, dodaje tuple (0, trenutni_karakter) u kompresovani izlaz.
- Nakon obrade poklapanja ili nepoklapanja, azuriramo indeks i prema potrebi
- Resenje u obluku tuple (1, pomeraj, duzina) ili (0, karakter)

#DEKODIRANJE
- Na samom pocetku iz binarnog fajla ocitava tuple (1, pomeraj, duzina) ili (0, karakter)
- Zatim vrsimo dekompresiju
- Ispisujemo rezultate

In [9]:
trenutni_direktoriju = os.getcwd()
putanjaLZ77 = os.path.join(trenutni_direktoriju, 'Ulazni_alfabet_LZ77.bin')

with open(putanjaLZ77, 'rb') as fajl:
        ulazni_alfabet = fajl.read()

def LZ77_kompresija(ulazni_niz, velicina_prozora):
    kompresovani_izlaz = []
    i = 0

    while i < len(ulazni_niz):
        maks_duzina_poklapanja = 0
        indeks_maks_poklapanja = 0

        # Provera maksimalnog poklapanja unazad do velicine_prozora
        for j in range(1, min(velicina_prozora + 1, i + 1)):
            duzina_poklapanja = 0
            while (i + duzina_poklapanja < len(ulazni_niz) and
                   ulazni_niz[i + duzina_poklapanja] == ulazni_niz[i - j + duzina_poklapanja]):
                duzina_poklapanja += 1
            if duzina_poklapanja > maks_duzina_poklapanja:
                maks_duzina_poklapanja = duzina_poklapanja
                indeks_maks_poklapanja = i - j

        if maks_duzina_poklapanja > 0:
            # Ako postoji poklapanje, dodajemo tuple (1, pomeraj, duzina) u izlaz
            kompresovani_izlaz.append((1, i - indeks_maks_poklapanja, maks_duzina_poklapanja))
            i += maks_duzina_poklapanja
        else:
            # Ako nema poklapanja, dodajemo tuple (0, trenutni_karakter) u izlaz
            kompresovani_izlaz.append((0, ulazni_niz[i]))
            i += 1

    return kompresovani_izlaz

#Sto je veci prozor vise kompresije se dobije ali sporije radi
velicina_prozora = 58
kompresovani_izlaz = LZ77_kompresija(ulazni_alfabet.decode('utf-8'), velicina_prozora)

def LZ77_upis_u_binarni_fajl(kompresovani_izlaz):
    with open("Kodiran_fajl_LZ77.bin","wb") as fajl:
        for element in kompresovani_izlaz:
            if element[0] == 0:
                # Ako nema poklapanja, zapisujemo (0, karakter)
                fajl.write(b'\x00')  # Oznaka za nema poklapanja
                fajl.write(element[1].encode('utf-8'))
            elif element[0] == 1:
                # Ako postoji poklapanje, zapisujemo (1, pomeraj, duzina)
                fajl.write(b'\x01')  # Oznaka za poklapanje
                fajl.write(element[1].to_bytes(2, byteorder='big'))  
                fajl.write(element[2].to_bytes(2, byteorder='big'))  


LZ77_upis_u_binarni_fajl(kompresovani_izlaz)

izlaz = []
    
with open("Kodiran_fajl_LZ77.bin", 'rb') as fajl:
    while True:
        marker = fajl.read(1)
        
        if not marker:
            break  # Kraj fajla
        
        if marker == b'\x00':
            # Nema poklapanja, čitamo karakter
            karakter = fajl.read(1).decode('utf-8')
            izlaz.append((0, karakter))
        elif marker == b'\x01':
            # Poklapanje, čitamo pomeraj i dužinu
            pomeraj = int.from_bytes(fajl.read(2), byteorder='big')
            duzina = int.from_bytes(fajl.read(2), byteorder='big')
            izlaz.append((1, pomeraj, duzina))



def lz77_dekodiranje(kodirani_podaci):
    dekodiran_niz = ""
    pom = ""

    for unos in kodirani_podaci:
        if unos[0] == 0:
            dekodiran_niz += unos[1]
            pom += unos[1]
        else:
            indeks_pocetka = len(pom) - unos[1]
            for i in range(unos[2]):
                dekodiran_niz += pom[indeks_pocetka + i]
                pom += pom[indeks_pocetka + i]

    return dekodiran_niz

dekompresovani_niz  = lz77_dekodiranje(izlaz)

ulazni_PrimerLZ77 = os.path.join(trenutni_direktoriju, 'Ulazni_alfabet_LZ77.bin')
dekodiran_PrimerLZ77 = os.path.join(trenutni_direktoriju, 'DekodiranLZ77.bin')

with open(dekodiran_PrimerLZ77, 'wb') as fajl:
    podaci = dekompresovani_niz.encode('utf-8')
    fajl.write(podaci)


if filecmp.cmp(ulazni_PrimerLZ77,dekodiran_PrimerLZ77):
    print("\nFajlovi imaju isti sadržaj.")
else:
    print("\nFajlovi nemaju isti sadržaj.")



Fajlovi imaju isti sadržaj.


<hr>

#3.2 LZW - KODIRANJE
- Na samom pocetku potrebno je sve ulazne simbole smestiti u recnik
- Krecemo od pocetka ulaznog alfabeta i trazimo da li postoji ponavljanje u recniku
- U slucaju da postoji, na izlaz saljemo indeks te pojave, a na recnik na poslednjem mestu dodajemo taj simbol + sledeci simbol
- Kod LZW ne preskacemo sledeci simbol koji smo dodali u recnik, vec sledeci je on na proveru
- Takodje je moguca pojava x uzastopnih simbola u recniku, ali postupak je isti
- Cuvamo u binarni fajl u prvoj liniji ulazne simbole, u drugoj izlazne indekse

#DEKODIRANJE
- Najpre smo ucitali prvi red, koji predstavlja pocetne karaktere recnika
- Ostalo su niz indeksa koja stizu i treba da se dekodiraju
- Na kraju sledi proces dekokiranja
- Upis resenja

In [10]:
trenutni_direktoriju = os.getcwd()
putanjaLZW = os.path.join(trenutni_direktoriju, 'Ulazni_alfabet_LZW.bin')

with open(putanjaLZW, 'rb') as fajl:
        ulazni_alfabet = fajl.read()

jedinstveni_simboli = []
def lzw_Koder(ulazni_niz):
    for x in ulazni_niz:
        if x not in jedinstveni_simboli:
            jedinstveni_simboli.append(x)

    # Pravimo recnik koji sadrži sve simbole koji se pojavljuju u ulaznom nizu
    recnik = {}
    for x in ulazni_niz:
        if x not in recnik:
            recnik[x] = len(recnik) + 1  # Dodajemo 1 jer počinjemo od 1, a ne od 0

    # Pravimo prazan niz kodova
    izlaz = []

    # Postavljamo trenutnu sekvencu na prvi simbol u ulaznom nizu
    trenutna_sekvencija = ulazni_niz[0]

    # Prolazimo kroz ostale simbole u ulaznom nizu
    for x in ulazni_niz[1:]:
        # Ako je trenutna sekvencija + x u rečniku
        if trenutna_sekvencija + x in recnik:
            # Postavljamo trenutnu sekvencu na trenutnu sekvencu + x
            trenutna_sekvencija = trenutna_sekvencija + x
        else:
            # Dodajemo kod trenutne sekvence u izlaz
            izlaz.append(recnik[trenutna_sekvencija])
            # Dodajemo trenutnu sekvencu + x u rečnik
            recnik[trenutna_sekvencija + x] = len(recnik) + 1
            # Postavljamo trenutnu sekvencu na x
            trenutna_sekvencija = x

    # Dodajemo kod trenutne sekvence u izlaz
    izlaz.append(recnik[trenutna_sekvencija])

    return izlaz, recnik


komprimiran_izlaz, komprimirani_recnik = lzw_Koder(ulazni_alfabet.decode('utf-8'))

# Prikaz rezultata
rezultatZaFajl = ''.join(str(broj) for broj in komprimiran_izlaz)
simboliZaFajl = ''.join(str(broj) for broj in jedinstveni_simboli)



niz1_string = ' '.join(map(str, jedinstveni_simboli))  # Pretvaranje u string sa razmacima
niz2_string = ' '.join(map(str, komprimiran_izlaz))
with open('Kodiran_fajl_LZW.bin', 'wb') as f:
    f.write(niz1_string.encode() + b'\n')  # Dodajemo novi red nakon prvog niza
    f.write(niz2_string.encode())  # Dodajemo novi red nakon prvog niza



with open('Kodiran_fajl_LZW.bin', 'rb') as file:
    # Čitanje prvog reda iz binarnog fajla
    prvi_red_bin = file.readline()
    drugi_red_bin = file.readline()

# Dekodiranje binarnih podataka u string koristeći utf-8
prvi_red_string = prvi_red_bin.decode('utf-8')
drugi_red_string = drugi_red_bin.decode('utf-8')


# Deljenje stringa na osnovu razmaka i pretvaranje u niz brojeva
simboli_LZW = [str(x) for x in prvi_red_string.split()]
indeksiLZW = [int(x) for x in drugi_red_string.split()]


def lzw_Dekoder(pocetni_simboli, niz_indeksa):
    recnik = {i + 1: simbol for i, simbol in enumerate(pocetni_simboli)}
    trenutni_indeks = len(recnik) + 1
    rezultat = [recnik[niz_indeksa[0]]]

    trenutna_sekvencija = recnik[niz_indeksa[0]]

    for indeks in niz_indeksa[1:]:
        if indeks in recnik:
            nova_sekvencija = recnik[indeks]
        elif indeks == trenutni_indeks:
            nova_sekvencija = trenutna_sekvencija + trenutna_sekvencija[0]
        
        rezultat.append(nova_sekvencija)

        # Dodajemo novu sekvenciju u rečnik
        recnik[trenutni_indeks] = trenutna_sekvencija + nova_sekvencija[0]
        trenutni_indeks += 1

        # Postavljamo trenutnu sekvencu na novu sekvencu
        trenutna_sekvencija = nova_sekvencija

    return ''.join(rezultat)

rezultat_dekodiranja = lzw_Dekoder(simboli_LZW, indeksiLZW)


print(f"Jedinstveni simboli: {jedinstveni_simboli}")

ulazni_PrimerLZW = os.path.join(trenutni_direktoriju, 'Ulazni_alfabet_LZW.bin')
dekodiran_PrimerLZW = os.path.join(trenutni_direktoriju, 'DekodiranLZW.bin')

with open(dekodiran_PrimerLZW, 'wb') as fajl:
    podaci = rezultat_dekodiranja.encode('utf-8')
    fajl.write(podaci)

if filecmp.cmp(ulazni_PrimerLZW,dekodiran_PrimerLZW):
    print("\nFajlovi imaju isti sadržaj.")
else:
    print("\nFajlovi nemaju isti sadržaj.")


Jedinstveni simboli: ['b', 'a', 'c']

Fajlovi imaju isti sadržaj.


<hr>

#Stepen kompresije
- Stepen kompresije = ulazni fajl / izlazni fajl

In [11]:
trenutni_direktoriju = os.getcwd()
putanja = os.path.join(trenutni_direktoriju, 'Ulazni_alfabet_Shanno_Fano_i_Huff.bin')
putanja2 = os.path.join(trenutni_direktoriju, 'Kodirani_fajl_shannon_fano.bin')
putanja3 = os.path.join(trenutni_direktoriju, 'Kodiran_fajl_huff.bin')
putanja4 = os.path.join(trenutni_direktoriju, 'Kodiran_fajl_LZ77.bin')

velicina_Ulaznog_fajla = os.path.getsize(putanja)
velicina_shannon_fano = os.path.getsize(putanja2)
velicina_huffmanov = os.path.getsize(putanja3)


stepen_kompresije_shannon_fano = velicina_Ulaznog_fajla / velicina_shannon_fano
stepen_kompresije_shannon_fano = round(stepen_kompresije_shannon_fano, 3)
print("Stepen kompresije za Shannon-Fano algoritama je: {}".format(stepen_kompresije_shannon_fano))

stepen_kompresije_velicina_huffmanov = velicina_Ulaznog_fajla / velicina_huffmanov
stepen_kompresije_velicina_huffmanov = round(stepen_kompresije_velicina_huffmanov, 3)
print("Stepen kompresije za Huffmanovog algoritama je: {}".format(stepen_kompresije_velicina_huffmanov))

putanja5 = os.path.join(trenutni_direktoriju, 'Ulazni_alfabet_LZ77.bin')
velicina_Ulaznog_fajla_LZ77 = os.path.getsize(putanja5)
velicina_LZ77 = os.path.getsize(putanja4)
stepen_kompresije_velicina_LZ77= velicina_Ulaznog_fajla_LZ77 / velicina_LZ77
stepen_kompresije_velicina_LZ77 = round(stepen_kompresije_velicina_LZ77, 3)
print("Stepen kompresije za LZ77 algoritama je: {}".format(stepen_kompresije_velicina_LZ77))

putanja6 = os.path.join(trenutni_direktoriju, 'Ulazni_alfabet_LZW.bin')
putanja7 = os.path.join(trenutni_direktoriju, 'Kodiran_fajl_LZW.bin')
velicina_Ulaznog_fajla_LZW = os.path.getsize(putanja6)
velicina_LZW = os.path.getsize(putanja7)
stepen_kompresije_velicina_LZW= velicina_Ulaznog_fajla_LZW / velicina_LZW
stepen_kompresije_velicina_LZW = round(stepen_kompresije_velicina_LZW, 3)
print("Stepen kompresije za LZW algoritama je: {}".format(stepen_kompresije_velicina_LZW))


Stepen kompresije za Shannon-Fano algoritama je: 3.586
Stepen kompresije za Huffmanovog algoritama je: 3.586
Stepen kompresije za LZ77 algoritama je: 1.237
Stepen kompresije za LZW algoritama je: 1.616
